# Backblaze Data Pipeline - S3 Setup and Data Loading

This notebook sets up an S3 bucket, downloads Backblaze hard drive data, and converts it to partitioned Parquet format for efficient querying.

## 1. Initialize S3 Bucket

Creates or retrieves an S3 bucket for storing raw and curated data. The bucket name is persisted in a `.env` file for reuse across sessions.

In [1]:
import os
import uuid
import boto3
from dotenv import load_dotenv, set_key

ENV_PATH = ".env"

# Load .env if it exists
load_dotenv(ENV_PATH)

bucket = os.getenv("BUCKET_NAME")

session = boto3.session.Session()
region = session.region_name
s3 = boto3.client("s3", region_name=region)

# If BUCKET_NAME exists, use it
if bucket:
    print("Using existing bucket from .env:", bucket)

else:
    # Create new bucket name
    bucket = f"mlops-backblaze-{uuid.uuid4().hex[:8]}-{region}"
    print("Creating new bucket:", bucket)

    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket)
    else:
        s3.create_bucket(
            Bucket=bucket,
            CreateBucketConfiguration={"LocationConstraint": region}
        )

    # Persist to .env
    if not os.path.exists(ENV_PATH):
        open(ENV_PATH, "w").close()

    set_key(ENV_PATH, "BUCKET_NAME", bucket)

print("Bucket ready:", bucket)

Using existing bucket from .env: mlops-backblaze-d7b30cb5-us-east-1
Bucket ready: mlops-backblaze-d7b30cb5-us-east-1


## 2. Create Folder Structure

Creates a standard MLOps folder structure in S3 to organize:
- Raw data (Backblaze hard drive stats, reviews)
- Curated/processed data
- Feature stores
- Model artifacts
- Evaluation results
- Monitoring data
- Batch inference outputs

In [ ]:
# prefixes = [
#     "raw/backblaze/",
#     "raw/reviews/",
#     "curated/",
#     "features/",
#     "artifacts/models/",
#     "artifacts/eval/",
#     "artifacts/monitoring/",
#     "inference/batch/"
# ]

# for p in prefixes:
#     s3.put_object(Bucket=bucket, Key=p)

# print("Folder layout created:")
# for p in prefixes:
#     print(" -", p)

Folder layout created:
 - raw/backblaze/
 - raw/reviews/
 - curated/
 - features/
 - artifacts/models/
 - artifacts/eval/
 - artifacts/monitoring/
 - inference/batch/


## 3. Verify Folder Structure

Lists all objects in the S3 bucket to confirm the folder structure was created successfully.

In [20]:
resp = s3.list_objects_v2(Bucket=bucket)
for obj in resp.get("Contents", []):
    print(obj["Key"])

artifacts/eval/
artifacts/models/
artifacts/monitoring/
curated/
features/
inference/batch/
raw/backblaze/
raw/backblaze/zips/data_Q1_2024.zip
raw/backblaze/zips/data_Q1_2025.zip
raw/backblaze/zips/data_Q2_2024.zip
raw/backblaze/zips/data_Q2_2025.zip
raw/backblaze/zips/data_Q3_2024.zip
raw/backblaze/zips/data_Q3_2025.zip
raw/backblaze/zips/data_Q4_2024.zip
raw/reviews/
raw/reviews_2023_parquet/raw_review_Electronics/part-000000.parquet


## 4. Download Backblaze Data to S3

Scrapes the Backblaze website to find all quarterly data files from 2024 onwards, then:
- Downloads each ZIP file directly from Backblaze
- Streams the data to S3 without storing locally (efficient for large files)
- Skips files that already exist in S3 to avoid redundant downloads
- Stores ZIP files in `raw/backblaze/zips/`

In [24]:
%%bash
set -e

BUCKET="mlops-backblaze-d7b30cb5-us-east-1"

curl -s https://www.backblaze.com/cloud-storage/resources/hard-drive-test-data \
  | grep -Eo 'https://f001\.backblazeb2\.com/file/Backblaze-Hard-Drive-Data/data_Q[1-4]_[0-9]{4}\.zip' \
  | grep -E '_202[5-9]\.zip' \
  | sort -u > backblaze_2025_plus_urls.txt

echo "Found URLs:"
wc -l backblaze_2025_plus_urls.txt
head -n 5 backblaze_2025_plus_urls.txt

while read -r url; do
  fname=$(basename "$url")

  # Skip if already in S3
  if aws s3 ls "s3://$BUCKET/raw/backblaze/zips/$fname" >/dev/null 2>&1; then
    echo "Skip: $fname"
    continue
  fi

  echo "Streaming upload: $fname"
  wget -qO- "$url" | aws s3 cp - "s3://$BUCKET/raw/backblaze/zips/$fname"
done < backblaze_2025_plus_urls.txt

echo "Done. Uploaded files in:"
aws s3 ls "s3://$BUCKET/raw/backblaze/zips/" | head


Found URLs:
3 backblaze_2025_plus_urls.txt
https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2025.zip
https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2025.zip
https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2025.zip
Skip: data_Q1_2025.zip
Skip: data_Q2_2025.zip
Skip: data_Q3_2025.zip
Done. Uploaded files in:
2026-01-25 13:42:10 1020483699 data_Q1_2025.zip
2026-01-25 13:43:05 1067562257 data_Q2_2025.zip
2026-01-25 13:44:03 1111587745 data_Q3_2025.zip


## 5. List Downloaded ZIP Files

Verifies that all ZIP files were successfully uploaded to S3.

In [25]:
%%bash
BUCKET="mlops-backblaze-d7b30cb5-us-east-1"
aws s3 ls "s3://$BUCKET/raw/backblaze/zips/"

2026-01-25 13:42:10 1020483699 data_Q1_2025.zip
2026-01-25 13:43:05 1067562257 data_Q2_2025.zip
2026-01-25 13:44:03 1111587745 data_Q3_2025.zip


## 6. List and Prepare ZIP Files for Processing

Imports libraries for data processing and lists all ZIP files from S3 that need to be converted to Parquet format. Uses pagination to handle large numbers of files efficiently.

In [26]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO
import zipfile

zip_s3_prefix = "raw/backblaze/zips/"
out_prefix = "curated/backblaze_parquet/"

# s3 = boto3.client("s3")

def list_s3_keys(prefix):
    paginator = s3.get_paginator("list_objects_v2")
    keys = []
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            if obj["Key"].endswith(".zip"):
                keys.append(obj["Key"])
    return sorted(keys)

zip_keys = list_s3_keys(zip_s3_prefix)
print("ZIPs found:", len(zip_keys))
print(zip_keys[:5])


ZIPs found: 3
['raw/backblaze/zips/data_Q1_2025.zip', 'raw/backblaze/zips/data_Q2_2025.zip', 'raw/backblaze/zips/data_Q3_2025.zip']


## 7. Convert CSV to Partitioned Parquet

Processes each ZIP file by:
1. Downloading the ZIP from S3 into memory
2. Extracting CSV files from the ZIP
3. Parsing date information from CSV filenames (YYYY-MM-DD)
4. Converting CSV to Parquet format with Snappy compression
5. Uploading to S3 with Hive-style partitioning: `year=YYYY/month=MM/day=DD/`
6. Skipping files that already exist to enable resumable processing

This partitioned structure enables efficient querying by date ranges in tools like Athena or Spark.

In [27]:
def upload_parquet_to_s3(df, s3_key):
    table = pa.Table.from_pandas(df, preserve_index=False)
    buf = BytesIO()
    pq.write_table(table, buf, compression="snappy")
    buf.seek(0)
    s3.put_object(Bucket=bucket, Key=s3_key, Body=buf.getvalue())

def check_s3_key_exists(s3_key):
    try:
        s3.head_object(Bucket=bucket, Key=s3_key)
        return True
    except:
        return False

for zip_key in zip_keys:
    print("Processing:", zip_key)
    zip_obj = s3.get_object(Bucket=bucket, Key=zip_key)["Body"].read()

    with zipfile.ZipFile(BytesIO(zip_obj), "r") as z:
        # Filter out macOS metadata files (._filename) and only get actual CSV files
        csv_files = [n for n in z.namelist() 
                     if n.endswith(".csv") and not os.path.basename(n).startswith("._")]

        for csv_name in csv_files:
            # csv_name is like 'data_Q1_2024/2024-01-01.csv'
            date_str = os.path.basename(csv_name).replace(".csv", "")
            yyyy, mm, dd = date_str.split("-")

            # Write to partitioned parquet key
            out_key = (
                f"{out_prefix}year={yyyy}/month={mm}/day={dd}/"
                f"{os.path.basename(zip_key).replace('.zip','')}_{dd}.parquet"
            )

            # Check if already extracted
            if check_s3_key_exists(out_key):
                print(f"  Skip (already exists): {out_key}")
                continue

            with z.open(csv_name) as f:
                df = pd.read_csv(f, encoding='utf-8', encoding_errors='replace')

            upload_parquet_to_s3(df, out_key)
            print(f"  Uploaded: {out_key}")

print("Done writing curated partitioned parquet to S3!")

Processing: raw/backblaze/zips/data_Q1_2025.zip
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=01/data_Q1_2025_01.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=02/data_Q1_2025_02.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=03/data_Q1_2025_03.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=04/data_Q1_2025_04.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=05/data_Q1_2025_05.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=06/data_Q1_2025_06.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=07/data_Q1_2025_07.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=08/data_Q1_2025_08.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=09/data_Q1_2025_09.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=10/data_Q1_2025_10.parquet
  Uploaded: curated/backblaze_parquet/year=2025/month=01/day=11/data_Q1_

In [ ]:
# %pip install -q -U datasets pyarrow s3fs pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
HF_DATASET = "McAuley-Lab/Amazon-Reviews-2023"

REVIEW_CONFIG = "raw_review_Electronics"
META_CONFIG   = "raw_meta_Electronics"

s3_reviews_out = f"s3://{bucket}/raw/reviews_2023_parquet/{REVIEW_CONFIG}/"
s3_meta_out    = f"s3://{bucket}/raw/reviews_2023_parquet/{META_CONFIG}/"

print(s3_reviews_out)
print(s3_meta_out)


s3://mlops-backblaze-d7b30cb5-us-east-1/raw/reviews_2023_parquet/raw_review_Electronics/
s3://mlops-backblaze-d7b30cb5-us-east-1/raw/reviews_2023_parquet/raw_meta_Electronics/


In [ ]:
from huggingface_hub import HfFileSystem
from pathlib import Path
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from io import BytesIO
import json

# Initialize HF filesystem
hf_fs = HfFileSystem()

dataset_name = HF_DATASET

# Find Electronics review file
print("Finding Electronics review file...")
review_path = f"datasets/{dataset_name}/raw/review_categories"
review_files = hf_fs.ls(review_path, detail=True)
electronics_files = [f for f in review_files if 'Electronics' in f['name']]

if not electronics_files:
    print("No Electronics files found")
else:
    file_info = electronics_files[0]
    file_path = file_info['name']
    file_name = Path(file_path).name
    
    print(f"Found: {file_name}")
    print(f"Size: {file_info['size'] / 1024 / 1024:.2f} MB")
    
    # First pass: Count total rows in JSONL file
    print(f"\nCounting total rows in JSONL file...")
    total_lines = 0
    with hf_fs.open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                total_lines += 1
            if total_lines % 500000 == 0:
                print(f"  Counted {total_lines:,} rows...")
    
    print(f"Total rows in JSONL file: {total_lines:,}\n")
    
    # Second pass: Stream JSONL from HF, convert to Parquet, upload directly to S3
    # This avoids filling up local disk
    # Filter for hard drive manufacturers: Western Digital, Toshiba, Seagate, Hitachi
    print(f"Streaming JSONL -> Parquet -> S3 (filtering for hard drive brands)...")
    
    # Define target brands for filtering (case-insensitive)
    TARGET_BRANDS = ['western digital', 'wd', 'toshiba', 'seagate', 'hitachi', 'hgst', 'wdc']
    
    def is_hard_drive_review(review_data):
        """Check if review is for Western Digital, Toshiba, Seagate, or Hitachi storage drives"""
        # Check in title, text, and parent_asin fields
        search_fields = []
        
        if 'title' in review_data and review_data['title']:
            search_fields.append(str(review_data['title']).lower())
        if 'text' in review_data and review_data['text']:
            search_fields.append(str(review_data['text']).lower())
        if 'parent_asin' in review_data:
            search_fields.append(str(review_data['parent_asin']).lower())
        
        # Combine all searchable text
        search_text = ' '.join(search_fields)
        
        # Check if any target brand appears in the text
        for brand in TARGET_BRANDS:
            if brand in search_text:
                # Additional check for storage-related keywords
                storage_keywords = ['hard drive', 'hdd', 'ssd', 'drive', 'storage', 'disk', 'external drive', 'internal drive']
                if any(keyword in search_text for keyword in storage_keywords):
                    return True
        return False
    
    chunk_size = 200_000
    part_idx = 0
    buffer = []
    total_rows = 0
    filtered_rows = 0
    
    # Open the JSONL file from HF
    with hf_fs.open(file_path, 'r') as f:
        for line_num, line in enumerate(f):
            if line.strip():
                review_data = json.loads(line)
                total_rows += 1
                
                # Filter for hard drive brands
                if is_hard_drive_review(review_data):
                    buffer.append(review_data)
                    filtered_rows += 1
                
                # When buffer reaches chunk_size, write to parquet and upload
                if len(buffer) >= chunk_size:
                    df = pd.DataFrame(buffer)
                    table = pa.Table.from_pandas(df, preserve_index=False)
                    
                    # Write to memory buffer
                    buf = BytesIO()
                    pq.write_table(table, buf, compression="snappy")
                    buf.seek(0)
                    
                    # Upload directly to S3
                    s3_key = s3_reviews_out.replace(f"s3://{bucket}/", "") + f"part-{part_idx:06d}.parquet"
                    s3.put_object(Bucket=bucket, Key=s3_key, Body=buf.getvalue())
                    
                    print(f"  Part {part_idx}: {len(buffer):,} rows → s3://{bucket}/{s3_key}")
                    
                    # Clear buffer
                    buffer = []
                    part_idx += 1
            
            # Progress update every 100k lines
            if (line_num + 1) % 100000 == 0:
                print(f"  Processed {line_num + 1:,} lines | Filtered: {filtered_rows:,}/{total_rows:,} ({100*filtered_rows/total_rows:.2f}%)")
    
    # Write remaining buffer
    if buffer:
        df = pd.DataFrame(buffer)
        table = pa.Table.from_pandas(df, preserve_index=False)
        
        buf = BytesIO()
        pq.write_table(table, buf, compression="snappy")
        buf.seek(0)
        
        s3_key = s3_reviews_out.replace(f"s3://{bucket}/", "") + f"part-{part_idx:06d}.parquet"
        s3.put_object(Bucket=bucket, Key=s3_key, Body=buf.getvalue())
        
        print(f"  Part {part_idx}: {len(buffer):,} rows → s3://{bucket}/{s3_key}")
    
    print(f"\n Complete!")
    print(f"   Total reviews processed: {total_rows:,}")
    print(f"   Hard drive reviews (WD/Toshiba/Seagate/Hitachi): {filtered_rows:,} ({100*filtered_rows/total_rows:.2f}%)")
    print(f"   Parquet files created: {part_idx + 1}")
    print(f"   S3 location: {s3_reviews_out}")


Finding Electronics review file...
Found: Electronics.jsonl
Size: 21568.52 MB

Counting total rows in JSONL file...
  Counted 500,000 rows...
  Counted 1,000,000 rows...
  Counted 1,500,000 rows...
  Counted 2,000,000 rows...
  Counted 2,500,000 rows...
  Counted 3,000,000 rows...
  Counted 3,500,000 rows...
  Counted 4,000,000 rows...
  Counted 4,500,000 rows...
  Counted 5,000,000 rows...
  Counted 5,500,000 rows...
  Counted 6,000,000 rows...
  Counted 6,500,000 rows...
  Counted 7,000,000 rows...
  Counted 7,500,000 rows...
  Counted 8,000,000 rows...
  Counted 8,500,000 rows...
  Counted 9,000,000 rows...
  Counted 9,500,000 rows...
  Counted 10,000,000 rows...
  Counted 10,500,000 rows...
  Counted 11,000,000 rows...
  Counted 11,500,000 rows...
  Counted 12,000,000 rows...
  Counted 12,500,000 rows...
  Counted 13,000,000 rows...
  Counted 13,500,000 rows...
  Counted 14,000,000 rows...
  Counted 14,500,000 rows...
  Counted 15,000,000 rows...
  Counted 15,500,000 rows...
  Coun

Checking S3 bucket structure...

1. Raw Backblaze ZIPs:
   raw/backblaze/zips/data_Q1_2024.zip                              959.88 MB
   raw/backblaze/zips/data_Q1_2025.zip                              973.21 MB
   raw/backblaze/zips/data_Q2_2024.zip                              973.38 MB
   raw/backblaze/zips/data_Q2_2025.zip                             1018.11 MB
   raw/backblaze/zips/data_Q3_2024.zip                             1000.73 MB

2. Curated Backblaze Parquet:
   curated/backblaze_parquet/year=._2024/month=01/day=01/data_Q1_2024_01.parquet       1044 bytes
   curated/backblaze_parquet/year=._2024/month=01/day=02/data_Q1_2024_02.parquet       1044 bytes
   curated/backblaze_parquet/year=._2024/month=01/day=03/data_Q1_2024_03.parquet       1044 bytes
   curated/backblaze_parquet/year=._2024/month=01/day=04/data_Q1_2024_04.parquet       1044 bytes
   curated/backblaze_parquet/year=._2024/month=01/day=05/data_Q1_2024_05.parquet       1044 bytes
   curated/backblaze_parquet/year

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2024-01-13,WD-WX31DB48X22V,WDC WD60EFRX,6001175126016,0,sac0,0,1002,0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-13,Z4D00WGP,ST6000DX000,6001175126016,0,sac0,0,1002,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-13,Z4D00YC6,ST6000DX000,6001175126016,0,sac0,0,1002,0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Read Sample Backblaze Parquet File from S3

Read a sample parquet file from the curated Backblaze data in S3.

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from io import BytesIO

# List files in the S3 path
s3_path = "curated/backblaze_parquet/"
response = s3.list_objects_v2(Bucket=bucket, Prefix=s3_path, MaxKeys=10)

if 'Contents' in response:
    # Get the first parquet file
    parquet_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.parquet')]
    
    if parquet_files:
        sample_file = parquet_files[0]
        print(f"Reading sample file: s3://{bucket}/{sample_file}")
        
        # Download and read the parquet file
        obj = s3.get_object(Bucket=bucket, Key=sample_file)
        buffer = BytesIO(obj['Body'].read())
        
        # Read into dataframe
        df_backblaze = pd.read_parquet(buffer)
        
        print(f"\nDataframe shape: {df_backblaze.shape}")
        print(f"\nColumns: {list(df_backblaze.columns)}")
        print(f"\nFirst few rows:")
        display(df_backblaze.head())
    else:
        print(f"No parquet files found in s3://{bucket}/{s3_path}")
else:
    print(f"No objects found in s3://{bucket}/{s3_path}")

Reading sample file: s3://mlops-backblaze-d7b30cb5-us-east-1/curated/backblaze_parquet/year=2025/month=01/day=01/data_Q1_2025_01.parquet

Dataframe shape: (304957, 197)

Columns: ['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'datacenter', 'cluster_id', 'vault_id', 'pod_id', 'pod_slot_num', 'is_legacy_format', 'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized', 'smart_2_raw', 'smart_3_normalized', 'smart_3_raw', 'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw', 'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_10_normalized', 'smart_10_raw', 'smart_11_normalized', 'smart_11_raw', 'smart_12_normalized', 'smart_12_raw', 'smart_13_normalized', 'smart_13_raw', 'smart_15_normalized', 'smart_15_raw', 'smart_16_normalized', 'smart_16_raw', 'smart_17_normalized', 'smart_17_raw', 'smart_18_normalized', 'smart_18_raw', 'smart_22_normalized', 'smart_22_raw', 'smart_23_normalized', '

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2025-01-01,2207E60CC65A,CT250MX500SSD1,250059350016,0,sac0,0,1028,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-01,2340E87B92B5,CT250MX500SSD1,250059350016,0,sac0,0,1028,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01,2EGK64VX,HGST HUH728080ALE604,8001563222016,0,sac0,0,1028,4,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-01,2EHZAKAX,HGST HUH728080ALE604,8001563222016,0,sac0,0,1028,12,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-01,2EJ02A1X,HGST HUH728080ALE604,8001563222016,0,sac0,0,1028,10,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Read Sample Reviews Parquet File from S3

Read a sample parquet file from the curated reviews data in S3.

In [3]:
# List files in the reviews S3 path
s3_reviews_path = "raw/reviews_2023_parquet/raw_review_Electronics/"
response_reviews = s3.list_objects_v2(Bucket=bucket, Prefix=s3_reviews_path, MaxKeys=10)

if 'Contents' in response_reviews:
    # Get the first parquet file
    reviews_parquet_files = [obj['Key'] for obj in response_reviews['Contents'] if obj['Key'].endswith('.parquet')]
    
    if reviews_parquet_files:
        sample_reviews_file = reviews_parquet_files[0]
        print(f"Reading sample reviews file: s3://{bucket}/{sample_reviews_file}")
        
        # Download and read the parquet file
        obj_reviews = s3.get_object(Bucket=bucket, Key=sample_reviews_file)
        buffer_reviews = BytesIO(obj_reviews['Body'].read())
        
        # Read into dataframe
        df_reviews = pd.read_parquet(buffer_reviews)
        
        print(f"\nDataframe shape: {df_reviews.shape}")
        print(f"\nColumns: {list(df_reviews.columns)}")
        print(f"\nFirst few rows:")
        display(df_reviews.head())
    else:
        print(f"No parquet files found in s3://{bucket}/{s3_reviews_path}")
else:
    print(f"No objects found in s3://{bucket}/{s3_reviews_path}")

Reading sample reviews file: s3://mlops-backblaze-d7b30cb5-us-east-1/raw/reviews_2023_parquet/raw_review_Electronics/part-000000.parquet

Dataframe shape: (171573, 10)

Columns: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']

First few rows:


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4.0,Great solid flash drive - Toshiba quality and ...,Not much can be said that hasn't been said in ...,[],B00IEZGU6G,B00WHEUS22,AFSKPY37N3C43SOI5IEXEK5JSIYA,1411650573000,0,False
1,5.0,GREAT External Drive,great price - I already had a Seagate 4TB an...,[],B01M00UHV8,B07454F4JH,AHGAOIZVODNHYMNCBV4DECZH42UQ,1494691925000,1,True
2,3.0,Long Term Life is Questionable; Seagate not go...,"The first one of these I got was DOA, but Seag...",[],B01IEKG3TY,B01IEKG3TY,AFAIJYOUO3NAWLBDIKTQSC3DASWA,1472494627000,1,False
3,4.0,"Compact Attractive, Quiet Unit",Several reviewers mentioned the difficulty in ...,[],B002HKEQZ6,B002OB4DC4,AFAIJYOUO3NAWLBDIKTQSC3DASWA,1306211962000,1,False
4,5.0,Works perfectly,I had a WD blue M.2 SSD that kept dropping off...,[],B09JCD2CN6,B09JCD2CN6,AFJBKPK5W56XWSNPQU2WW66ISWYQ,1640360340432,0,False


In [4]:
# Count unique ASIN values
unique_asin_count = df_reviews['asin'].nunique()
print(f"Unique ASIN values: {unique_asin_count:,}")

# Also show unique parent_asin if it exists
if 'parent_asin' in df_reviews.columns:
    unique_parent_asin_count = df_reviews['parent_asin'].nunique()
    print(f"Unique parent_asin values: {unique_parent_asin_count:,}")

Unique ASIN values: 45,735
Unique parent_asin values: 40,965


## Examine Data for Joining

Check sample values from both datasets to understand how to create a join key.

In [5]:
# Sample review titles and parent_asin to see product names
print("Sample Reviews Data:")
print(df_reviews[['parent_asin', 'title', 'text']].head(10))

print("\n" + "="*80 + "\n")

# Sample backblaze model names
print("Sample Backblaze Model Data:")
print(df_backblaze['model'].value_counts().head(20))

Sample Reviews Data:
  parent_asin                                              title  \
0  B00WHEUS22  Great solid flash drive - Toshiba quality and ...   
1  B07454F4JH                               GREAT External Drive   
2  B01IEKG3TY  Long Term Life is Questionable; Seagate not go...   
3  B002OB4DC4                     Compact Attractive, Quiet Unit   
4  B09JCD2CN6                                    Works perfectly   
5  B0C6QYPNYR                               Great for mobile use   
6  B01FRP1ZHE                           Chic looking hard drive.   
7  B01BCWKBZI                                    Fast and small.   
8  B09VS4V18K                                                DOA   
9  B018AX3OHO  This thing is even faster than my Macbook than...   

                                                text  
0  Not much can be said that hasn't been said in ...  
1  great price  -  I already had a Seagate 4TB an...  
2  The first one of these I got was DOA, but Seag...  
3  Several

## Create Join Keys

Extract manufacturer and model information from reviews to match with Backblaze model data.

In [6]:
import re

def extract_manufacturer(text):
    """Extract manufacturer from review text"""
    text_lower = str(text).lower()
    
    # Map various manufacturer names to standard format
    if 'toshiba' in text_lower:
        return 'TOSHIBA'
    elif 'seagate' in text_lower or 'st' in text_lower[:3]:
        return 'SEAGATE'
    elif 'western digital' in text_lower or 'wd' in text_lower or 'wdc' in text_lower:
        return 'WDC'
    elif 'hitachi' in text_lower or 'hgst' in text_lower:
        return 'HGST'
    
    return None

def extract_model_hints(text):
    """Extract potential model numbers/patterns from review text"""
    text_upper = str(text).upper()
    
    # Look for common model patterns
    patterns = [
        r'[A-Z]{2,}\s*[A-Z0-9]{6,}',  # e.g., "MG08ACA16TA", "WUH722222ALE6L4"
        r'ST\d{4,}[A-Z]{2}\d{3,}[A-Z]?',  # Seagate pattern
        r'WD[A-Z0-9]{6,}',  # WD pattern
        r'MG\d{2}[A-Z]{3}\d{2}[A-Z]{2,}',  # Toshiba pattern
    ]
    
    models = []
    for pattern in patterns:
        matches = re.findall(pattern, text_upper)
        models.extend(matches)
    
    return models if models else None

# Add manufacturer and model hints to reviews dataframe
df_reviews['manufacturer'] = df_reviews['title'].apply(extract_manufacturer)
df_reviews['manufacturer'] = df_reviews['manufacturer'].fillna(
    df_reviews['text'].apply(extract_manufacturer)
)

df_reviews['model_hints'] = df_reviews['title'].apply(extract_model_hints)
df_reviews['model_hints'] = df_reviews['model_hints'].fillna(
    df_reviews['text'].apply(extract_model_hints)
)

# Add manufacturer to backblaze data for easier joining
def get_bb_manufacturer(model):
    """Extract manufacturer from Backblaze model name"""
    if model.startswith('TOSHIBA'):
        return 'TOSHIBA'
    elif model.startswith('ST') or model.startswith('SEAGATE'):
        return 'SEAGATE'
    elif model.startswith('WDC') or model.startswith('WD'):
        return 'WDC'
    elif model.startswith('HGST') or model.startswith('HITACHI'):
        return 'HGST'
    return None

df_backblaze['manufacturer'] = df_backblaze['model'].apply(get_bb_manufacturer)

print("Reviews with manufacturer extracted:")
print(df_reviews[['parent_asin', 'title', 'manufacturer', 'model_hints']].head(10))
print(f"\nReviews with manufacturer identified: {df_reviews['manufacturer'].notna().sum():,} / {len(df_reviews):,}")

print("\n" + "="*80 + "\n")

print("Backblaze with manufacturer extracted:")
print(df_backblaze[['model', 'manufacturer']].head(10))
print(f"\nManufacturer distribution in Backblaze:")
print(df_backblaze['manufacturer'].value_counts())

Reviews with manufacturer extracted:
  parent_asin                                              title manufacturer  \
0  B00WHEUS22  Great solid flash drive - Toshiba quality and ...      TOSHIBA   
1  B07454F4JH                               GREAT External Drive      SEAGATE   
2  B01IEKG3TY  Long Term Life is Questionable; Seagate not go...      SEAGATE   
3  B002OB4DC4                     Compact Attractive, Quiet Unit      SEAGATE   
4  B09JCD2CN6                                    Works perfectly          WDC   
5  B0C6QYPNYR                               Great for mobile use          WDC   
6  B01FRP1ZHE                           Chic looking hard drive.          WDC   
7  B01BCWKBZI                                    Fast and small.          WDC   
8  B09VS4V18K                                                DOA          WDC   
9  B018AX3OHO  This thing is even faster than my Macbook than...          WDC   

                                         model_hints  
0               

## Join Strategy Summary

Now both datasets have a `manufacturer` field that can be used for joining:

- **Reviews**: `df_reviews['manufacturer']` - extracted from title/text (TOSHIBA, SEAGATE, WDC, HGST)
- **Backblaze**: `df_backblaze['manufacturer']` - extracted from model name

You can join on manufacturer to analyze reviews by manufacturer against Backblaze failure data.

In [7]:
# Example join: Get reviews by manufacturer
print("Join Example - Reviews by Manufacturer:")
print("\nReviews distribution:")
print(df_reviews['manufacturer'].value_counts())

print("\n" + "="*80 + "\n")

print("Backblaze distribution:")
print(df_backblaze['manufacturer'].value_counts())

# You can now join like this:
# joined_df = df_reviews.merge(df_backblaze, on='manufacturer', how='inner')
# Or do aggregations by manufacturer before joining

Join Example - Reviews by Manufacturer:

Reviews distribution:
manufacturer
WDC        103758
SEAGATE     40251
TOSHIBA     16134
HGST         3263
Name: count, dtype: int64


Backblaze distribution:
manufacturer
SEAGATE    105834
TOSHIBA     95959
WDC         68377
HGST        31654
Name: count, dtype: int64
